In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import glob

import pandas as pd

In [8]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_692780/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_692780/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_692780/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_692780/2803701148.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs
0,\AKROUSDT\plots\2022-03-07T074200\backtest_res...,5.5,AKROUSDT,True,17.780,0.0766,100.0,0.9507,True,19.350,0.0738,100.0,0.9633
1,\FLMUSDT\plots\2022-03-07T075107\backtest_resu...,5.5,FLMUSDT,True,7.212,0.0333,100.0,0.9352,True,22.120,0.0928,100.0,0.9539
2,\GRTUSDT\plots\2022-02-23T153035\backtest_resu...,5.4,GRTUSDT,True,14.060,0.0654,100.0,0.9689,True,9.442,0.0456,100.0,0.9847
3,\GRTUSDT\plots\2022-02-25T222300\backtest_resu...,5.5,GRTUSDT,True,15.000,0.0696,100.0,0.9691,True,9.567,0.0462,100.0,0.9847
4,\GRTUSDT\plots\2022-02-25T232504\backtest_resu...,5.5,GRTUSDT,True,15.000,0.0696,100.0,0.9691,True,9.567,0.0462,100.0,0.9847
...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,\ZECUSDT\plots\2022-02-25T232208\backtest_resu...,5.5,ZECUSDT,True,12.520,0.0599,100.0,0.9660,True,3.435,0.0173,100.0,0.9792
390,\ZENUSDT\plots\2022-03-05T072906\backtest_resu...,5.5,ZENUSDT,True,8.481,0.0372,100.0,0.9209,True,19.940,0.0882,100.0,0.8434
391,\ZILUSDT\plots\2022-03-06T080955\backtest_resu...,5.5,ZILUSDT,True,12.050,0.0533,100.0,0.9544,True,15.840,0.0657,100.0,0.9504
392,\ZRXUSDT\plots\2022-03-06T230914\backtest_resu...,5.5,ZRXUSDT,True,15.580,0.0683,100.0,0.9472,True,14.480,0.0616,100.0,0.9139


Now filter the dataframe

In [9]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = ["ZRXUSDT","SCUSDT","AKROUSDT","XEMUSDT"]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['short'] == True)
              & (big_data['adgt_%'] > 0.1)
              & (big_data['bnkrl_%'] > 95.0)
              & (big_data['bnkrs_%'] > 95.0)
              & (big_data['ebrs'] > .85)
              & (big_data['ebrl'] > .85)
              & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs,adgt_%
0,\AKROUSDT\plots\2022-03-07T074200\backtest_res...,5.5,AKROUSDT,True,17.78,0.0766,100.0,0.9507,True,19.35,0.0738,100.0,0.9633,0.1504
254,\SCUSDT\plots\2022-03-07T073726\backtest_resul...,5.5,SCUSDT,True,7.34,0.0345,100.0,0.9430,True,18.24,0.0749,100.0,0.9264,0.1094
392,\ZRXUSDT\plots\2022-03-06T230914\backtest_resu...,5.5,ZRXUSDT,True,15.58,0.0683,100.0,0.9472,True,14.48,0.0616,100.0,0.9139,0.1299
393,\ZRXUSDT\plots\2022-03-07T073116\backtest_resu...,5.5,ZRXUSDT,True,19.07,0.0824,100.0,0.9488,True,17.13,0.0733,100.0,0.9169,0.1557


In [11]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_692780/2043545472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
0,\AKROUSDT\plots\2022-03-07T074200\backtest_res...,AKROUSDT,0.1504,0.0766,0.0738,100.0,100.0,0.9507,0.9633
254,\SCUSDT\plots\2022-03-07T073726\backtest_resul...,SCUSDT,0.1094,0.0345,0.0749,100.0,100.0,0.9430,0.9264
392,\ZRXUSDT\plots\2022-03-06T230914\backtest_resu...,ZRXUSDT,0.1299,0.0683,0.0616,100.0,100.0,0.9472,0.9139
393,\ZRXUSDT\plots\2022-03-07T073116\backtest_resu...,ZRXUSDT,0.1557,0.0824,0.0733,100.0,100.0,0.9488,0.9169
